In [1]:
#@markdown **Double click here to see the code to import required libraries and the dataset**

#@markdown To play around with the graphs, execute the cells in order (ctrl + enter), starting with this one

#@markdown You will be prompted to give your credentials to access the data spreadsheet

#@markdown This notebook includes the code to
#@markdown * Visualize the **contour plot of the generalized logical qubit metric** 
#@markdown * **visualize the data**
#@markdown * perform a **multivariate loglinear regression with bootstrapping** 
#@markdown * perform a **multivariate extrapolation with bootstrapping**
#@markdown * extrapolate via a **geometric drift model** 

#@markdown Double click any cell to see the associated code.


import numpy as np
from scipy import special
from scipy import stats
from scipy import odr
from scipy.stats import multivariate_normal
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import math
import pandas as pd
import altair as alt
import collections
from datetime import datetime


from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

# worksheet = gc.open('Quantum computing progress').sheet1
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1pwb4gf0FxlxgfVhtXTaqEGS9b7FwsstsJ0v7Zb1naQ0').sheet1
# worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1utGUc1Okg4XfF06Ilhyy0jFLqL9RnbFaVXDynwFjfb8/edit#gid=0').sheet1


# get_all_records gives a dict with the contents of the spreadsheet
rows = worksheet.get_all_records()

# Convert to a DataFrame
df = pd.DataFrame.from_records(rows)

# Fix metric column types
metrics = ["Physical qubits", "Average two-qubit gate error rate"]
for metric in metrics:
  df[metric] = pd.to_numeric(df[metric], errors="coerce")

df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Impute missing dates with year when available
df["Date"] = df.apply(lambda row : row["Date"] \
              if row["Date"] is not pd.NaT or pd.isna(row["Year"]) \
              else pd.Timestamp(year=np.int64(row["Year"]), month=6, day=1),
              axis=1)

# (discretization) Rounds dates down to the year
#df["Date"] =  df.apply(lambda row : pd.Timestamp(year=np.int64(row["Date"].year), month=1, day=1) if not pd.isna(row["Date"]) else np.nan, axis=1)

# Substitute year with fractional year, rounded to the month
df["Year"] = df.apply(lambda row : row["Date"].year + row["Date"].month/12, axis=1)

#######################################################################
# show all data
df

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,Designation,Source,Link to source,Meta source,Group,Source type,Technology,Date,Year,Physical qubits,Average two-qubit gate error rate,NOTE: Hover over the cells marked with a black triangle for context and quotes,,"THIS DATASET IS LICENSED UNDER CC BY 4.0. \nPLEASE CITE AS Jaime Sevilla and C. Jess Riedel. ""Forecasting timelines of quantum computing"", 2020"
0,Aspen-7,"""Proveedores de hardware de Amazon Braket""",https://aws.amazon.com/es/braket/hardware-prov...,Quantum Computing Report,Rigetti,Hardware provider information sheet,Superconductor,2019-10-24,2019.833333,32,0.04800,,,
1,Yorktown - ibmqx2,(IBM Quantum Computing Experience),https://quantum-computing.ibm.com/,IBM Quantum Experience Available Systems,IBM,Cloud quantum computing service,Superconductor,2019-03-15,2019.250000,5,0.03960,,,
2,Melbourne,(IBM Quantum Computing Experience),https://quantum-computing.ibm.com/,IBM Quantum Experience Available Systems,IBM,Cloud quantum computing service,Superconductor,2018-12-21,2019.000000,15,0.05450,,,
3,Vigo,(IBM Quantum Computing Experience),https://quantum-computing.ibm.com/,IBM Quantum Experience Available Systems,IBM,Cloud quantum computing service,Superconductor,2019-02-08,2019.166667,5,0.00966,,,
4,Ourense,(IBM Quantum Computing Experience),https://quantum-computing.ibm.com/,IBM Quantum Experience Available Systems,IBM,Cloud quantum computing service,Superconductor,2019-02-08,2019.166667,5,0.00909,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,NA,State preservation by repetitive error detecti...,https://www.nature.com/articles/nature14270,N/A,Google,Journal article,Superconductor,2015-04-03,2015.333333,9,0.41000,,,
59,NA,Superconducting quantum circuits at the surfac...,https://www.nature.com/articles/nature13171,N/A,University of California,Journal article,Superconductor,2014-04-23,2014.333333,5,0.00600,,,
60,Aspen-4,,https://quantumcomputingreport.com/scorecards/...,Quantum Computing Report,Rigetti,[NEED BETTER SOURCE],Superconductor,2019-05-30,2019.416667,16,0.09650,,,
61,Cambridge,,https://quantum-computing.ibm.com/docs/cloud/b...,IBM Quantum Experience Available Systems,IBM,[COMING SOON],Superconductor,NaT,NaN,28,NaN,,,


# Data visualization

In [2]:
#@title Plot key metrics against years {display-mode: "form"}
target = "Physical qubits" #@param ["Physical qubits", "Average two-qubit gate error rate"]
data_horizon = [2002, 2022] #@param
data_horizon = [pd.Timestamp(year=x, month=1, day=1) for x in data_horizon]

# Filter the data
df_hat = df.dropna(subset=["Date", target])
df_hat = df_hat[df_hat[target]> 0.]
df_hat = df_hat[data_horizon[0] <= df_hat["Date"]]
df_hat = df_hat[df_hat["Date"] <= data_horizon[1]]

print(f"n = {len(df_hat)}")

# Plot the filtered data
alt.Chart(df_hat,width=1000, height=800
).mark_point(size=200).encode(
  x=alt.X('Date', scale=alt.Scale(), axis = alt.Axis(format = ("%Y"))),
  y=alt.Y(target, scale=alt.Scale(type="log"),axis=alt.Axis(format=".1e" if target != "Physical qubits" else "i")),
  shape = 'Technology',
  tooltip='Source'
).configure_axis(grid=False,
    labelFontSize=20,
    titleFontSize=40).interactive().configure_legend(
    titleFontSize=20,
    labelFontSize = 18,
    gradientLength=400,
    gradientThickness=30
)

n = 52


alt.Chart(...)

In [ ]:
#@title Plot physical qubits against error rate {display-mode: "form"}
mask = df.notna()["Physical qubits"]
mask &= df.notna()["Average two-qubit gate error rate"]
n = np.sum(mask)
print(f"n={ n }")
alt.Chart(df, width=1000, height=800).mark_point(size=200).encode(
  x=alt.X("Physical qubits", scale=alt.Scale(type="log")),
  y=alt.Y("Average two-qubit gate error rate", scale=alt.Scale(type="log"),axis=alt.Axis(format=".1e")),
  color = alt.Color("Year:O", 
                    scale=alt.Scale(scheme='viridis')
          ),
  shape = 'Technology'
).configure_axis(grid=False,
    labelFontSize=20,
    titleFontSize=40).interactive().configure_legend(
    titleFontSize=20,
    labelFontSize = 18,
    gradientLength=400,
    gradientThickness=30
)

n=40


alt.Chart(...)